# **LSTMVAE**
# Вариационный автоэнкодер на основе LSTM блоков для генерации временных рядов

# 01. Подключение библиотек, настройка

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly import subplots
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'facecolor' : '#D9D9D9'}
sns.set(font_scale=1.3)

In [ ]:
import sklearn.model_selection
from sklearn.metrics import mean_absolute_error, median_absolute_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

Для создания архитектуры вариационного автоэнкодера воспользуемся пакетом Keras.

In [ ]:
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, RepeatVector, Dropout, BatchNormalization
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam
from keras import objectives
import tensorflow as tf

Для обучения нейронной сети будем использовать Google Colab.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Osipov_NO_test_task_2_chemtechai_data_analyst

Mounted at /content/drive
/content/drive/My Drive/Osipov_NO_test_task_2_chemtechai_data_analyst


Проверим подключение к GPU Google Colab.

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9908405011686310669
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14674281152
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8512319098609485681
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


Настроим зерно датчика случайных чисел.

In [ ]:
def set_seed(seed):
    """Функция устанавливающая зерно датчика случайных чисел"""
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
set_seed(1)

# 02. Подключение данных

In [ ]:
data = pd.read_csv("data_pi.csv")

In [ ]:
data.head()

,timestamp,PI_71
0,2020-04-27 15:30:00,272.0
1,2020-04-27 16:30:00,272.0
2,2020-04-27 19:30:00,272.0
3,2020-04-27 20:30:00,272.0
4,2020-04-27 21:30:00,272.0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60510 entries, 0 to 60509
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  60510 non-null  object 
 1   PI_71      60510 non-null  float64
dtypes: float64(1), object(1)
memory usage: 945.6+ KB


Описательные статистики.

In [ ]:
data.describe()

,PI_71
count,60510.000000
mean,279.636115
std,76.820794
min,70.023150
25%,244.502300
50%,278.790500
75%,321.759200
max,756.221100


In [ ]:
data.isnull().sum()

timestamp    0
PI_71        0
dtype: int64

Переименуем столбцы для удобства.

In [ ]:
data.columns = ["timestamp", "marks"]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60510 entries, 0 to 60509
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  60510 non-null  object 
 1   marks      60510 non-null  float64
dtypes: float64(1), object(1)
memory usage: 945.6+ KB


# 03. Визуализация данных

Визуализируем данные с помощью библиотеки Plotly

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data["timestamp"], 
                         y=data["marks"], 
                         name='PI_71 marks'))
fig.update_layout(showlegend=True, 
                  title="PI_71 Marks", 
                  xaxis_title="Time", 
                  yaxis_title="Marks")
fig.show()

Output hidden; open in https://colab.research.google.com to view.

По графику видно, что во временном ряду присутствуют пропущенные значения. 
Возможно это связано с особенностями производства.

Посмотрим на распределение значений.

In [ ]:
fig = ff.create_distplot([data["marks"]], 
                         group_labels=["PI_71 Marks"], 
                         colors=["#636EFA"])
fig.update_layout(title_text='PI_71 Distribution')
fig.show()

Output hidden; open in https://colab.research.google.com to view.

# 04. Обработка данных

Преобразуем столбец timestamp к типу datetime.

In [ ]:
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60510 entries, 0 to 60509
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  60510 non-null  datetime64[ns]
 1   marks      60510 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 945.6 KB


Столбец timestamp сохраним в переменную timestamps, для удобства.

Также создадим DataFrame со столбцом значений PI_71.

In [ ]:
timestamps = data["timestamp"]
data_marks = data.drop('timestamp', axis=1)
data_marks.head()

,marks
0,272.0
1,272.0
2,272.0
3,272.0
4,272.0


Стандартизируем данные. 

Переведем в значения масштаб значений со средним $\mu = \mathrm{0}$ и стандартным отклонением $\sigma = \mathrm{1}$, по формуле $z = {x - \mu \over \sigma}$. 

Также сохраним в переменных min_x, max_x, mean_x, stddev_x  - минимальное и максимальное значения, среднее и стандартное отклонение для возврата к оригинальному масштабу.

In [ ]:
min_x = data.iloc[:, 1].min()
max_x = data.iloc[:, 1].max()
mean_x = data.iloc[:, 1].mean()
stddev_x = data.iloc[:, 1].std()

std_scaler = StandardScaler()
data_marks["marks"] = std_scaler.fit_transform(data_marks)

In [ ]:
data_marks

,marks
0,-0.099402
1,-0.099402
2,-0.099402
3,-0.099402
4,-0.099402
...,...
60505,0.625550
60506,0.629316
60507,0.629316
60508,0.631200


Разделим временной ряд на train (60%) и test (40%) выборки.

Добавим в train выборку дополнительно 24 значения. Это не создаст большого дисбаланса в размерах тренировочной и тестовой выборки, но позволит нам подобрать параметр batch size из более широкого диапазона значений. 

Batch size должен быть кратен размерам тренировочной и тестовой выборки.

In [ ]:
train_size = int(data_marks.shape[0] * 0.6) + 24

train = pd.DataFrame(data_marks.iloc[0:train_size, :])
test = pd.DataFrame(data_marks.iloc[train_size:, :])

Создадим тензоры размерности 3, для эффективной работы LSTM блоков. Тензор будет иметь размерность (samples, timesteps, features).

Объявим переменную timesteps со значением 30.

У нас получится вложенный список, в котором для каждого значения будут сохранены вектора со следующими 30 значениями.

In [ ]:
timesteps = 30

def create_sequences(x, time_steps=timesteps):
    """Функция для создания тензора размерностью:
    (len(x), timesteps, features)
    x : временной ряд
    """
    x_seq = []
    for i in range(len(x) - time_steps):
        x_seq.append(x.iloc[i:(i + time_steps)].values)
    return np.array(x_seq)

X_train = create_sequences(train[["marks"]])
X_test = create_sequences(test[['marks']])

print(X_train.shape, X_test.shape, sep="\n")

(36300, 30, 1)
(24150, 30, 1)


Подберем размер batch size. Batch size значение должно быть кратно размерам тренировочной и тестовой выборки.

In [ ]:
def get_batch_size(train, test):
    """Функция выдающая на печать диапазон значений batch size
    train : тренировочная выборка
    test : тестовая выборка    
    """
    
    def isInt(value):
        """Функция для проверки, является ли число целым
        value : значение
        """
        return int(value) == float(value)
    
    
    rows_list = []
    for i in range(1, 128):
        result_train = train.shape[0] / i
        result_test = test.shape[0] / i
        if isInt(result_train) and isInt(result_test):
            rows_list.append([i, int(result_train), int(result_test)])
            
    output_df = pd.DataFrame(rows_list, columns=["batch_size", 
                                                 "train_size", 
                                                 "test_size"],)
    print(output_df)

In [ ]:
get_batch_size(X_train, X_test)

    batch_size  train_size  test_size
0            1       36300      24150
1            2       18150      12075
2            3       12100       8050
3            5        7260       4830
4            6        6050       4025
5           10        3630       2415
6           15        2420       1610
7           25        1452        966
8           30        1210        805
9           50         726        483
10          75         484        322


Ниже при объявлении переменных используемых в архитектуре нейронной сети выберем batch size равным 75.

# 05. Реализация архитектуры вариационного автоэнкодера (VAE) с LSTM блоками

Создадим функции необходимые внутри архитектуры нейронной сети.
- функция сэмплирования распределения
- функция для борьбы с переобучением, задающая граф с двумя слоями: Dropout, BatchNormalization

In [ ]:
def sampling(args):
    """Функция сэмплирования, для реализации логики вариационного автоэнкодера.
    Используется в Lambda-слое VAE.
    args : список аргументов, функция будет принимать 
    выходные данные слоев z_mean и z_log_var
    """
    global batch_size, latent_dim
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), 
                              mean=0., 
                              stddev=1.)    
    return z_mean + K.exp(z_log_var / 2) * epsilon

def dropout_and_batchnorm(x):
    """Функция добавляющая слои Dropout и BatchNormalization.
    Используется для снижения переобучения.
    x : выход предыдущего слоя
    """
    global dropout_rate
    return Dropout(dropout_rate)(BatchNormalization()(x))

Используем слудющие параметры описывающие архитектуру вариационного автоэнкодера:
- Размеры слоев (указаны ниже в переменных с обозначением _dim)
- Активационная функция: $tanh(x) =  \frac {e^{x} - e^{-x}} {e^{x} + e^{-x}}$ (гиперболический тангенс)
- Размер batch
- Значения dropout rate, learning rate
- Количество эпох обучения

Ниже объявим переменные необходимые для описания архитектуры вариационного автоэнкодера.

In [ ]:
# размеры слоев
original_dim = X_train.shape[-1]
first_dim = 256
second_dim = 128
third_dim = 64
latent_dim = 15

# настраиваемые параметры для обучения
act_func = "tanh"
batch_size = 75
dropout_rate = 0.2
learning_rate = 1e-4
epochs = 100

# регуляризация
# alpha = 1e-4
# regularizer = keras.regularizers.l2(alpha)

Архитектура вариационного автоэнкодера.

Энкодер:
- Входной слой
- 3 рекуррентных LSTM блока
- Слои Dropout и BatchNormalization для ускорения обучения и снижения переобучения
- 2 независимых полносвязных слоя описывающие среднее и дисперсию распределения

Скрытый слой:
- Lamda слой сэмплирующий из нормального распределения $N$ со средним 0 и дисперсией 1

Декодер:
- Входной слой декодера добавляющий необходимую размерность
- 3 рекурентных LSTM блока
- Выходной рекурентный LSTM блок

Ниже создадим архитектуру VAE.

In [ ]:
# Энкодер с LSTM блоками
input_layer = Input(batch_shape=(batch_size, timesteps, original_dim))
x = LSTM(first_dim, return_sequences=True, activation=act_func)(input_layer)
x = dropout_and_batchnorm(x)
x = LSTM(second_dim, return_sequences=True, activation=act_func)(x)
x = dropout_and_batchnorm(x)
x = LSTM(third_dim, activation=act_func, )(x)

# Скрытый Lambda слой, предсказывающий параметры распределений: 
# z_mean и z_log_var (среднее и логарифм дисперсии) 
# находящиеся на двух независимых полносвязных слоях
z_mean = Dense(latent_dim)(x)
z_log_var = Dense(latent_dim)(x)
z = Lambda(sampling, output_shape=(latent_dim, ))([z_mean, z_log_var])

# Декодер с LSTM блоками
input_decoder = RepeatVector(timesteps)(z)
h = LSTM(third_dim, return_sequences=True, activation=act_func)(input_decoder)
h = dropout_and_batchnorm(h)
h = LSTM(second_dim, return_sequences=True, activation=act_func)(h)
h = dropout_and_batchnorm(h)
h = LSTM(first_dim, return_sequences=True, activation=act_func)(h)
output_layer = LSTM(original_dim, return_sequences=True)(h)

# Создание модели VAE
vae = Model(input_layer, output_layer, name="VAE")

In [ ]:
# Реализация дивергенции Кульбака-Лейблера
# xent_loss = objectives.mse(input_layer, output_layer)
# kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var))
# vae_loss = K.mean(xent_loss + kl_loss)
# vae.add_loss(vae_loss)

Для оптимизации будем использовать loss - средняя квадратичная ошибка (mean squared error), из пакета Keras.

In [ ]:
vae_loss_mse = objectives.mse(input_layer, output_layer)
vae.add_loss(vae_loss_mse)

В качестве оптимизатора будем использовать Adam со следующими параметрами:

In [ ]:
adam_tune = Adam(learning_rate=learning_rate, 
                 beta_1=0.9, 
                 beta_2=0.999, 
                 epsilon=1e-08, 
                 decay=0.0)

Компиляция нейронной сети. При обучении, помимо значения loss, будем выводить значение mean absolute error.

In [ ]:
vae.compile(optimizer=adam_tune, metrics=[keras.metrics.MAE])
vae.summary()

Model: "VAE"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(75, 30, 1)]        0                                            
__________________________________________________________________________________________________
lstm_21 (LSTM)                  (75, 30, 256)        264192      input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_12 (BatchNo (75, 30, 256)        1024        lstm_21[0][0]                    
__________________________________________________________________________________________________
dropout_12 (Dropout)            (75, 30, 256)        0           batch_normalization_12[0][0]     
________________________________________________________________________________________________

# 06. Обучение модели, прогнозирование

Обучим модель с помощью функции fit

In [ ]:
vae.fit(X_train, X_train, epochs=epochs, batch_size=batch_size)

Epoch 1/100
484/484 [==============================] - 47s 19ms/step - loss: 0.1618 - mean_absolute_error: 0.2154
Epoch 2/100
484/484 [==============================] - 9s 19ms/step - loss: 0.0988 - mean_absolute_error: 0.1319
Epoch 3/100
484/484 [==============================] - 9s 19ms/step - loss: 0.0943 - mean_absolute_error: 0.1202
Epoch 4/100
484/484 [==============================] - 9s 19ms/step - loss: 0.0876 - mean_absolute_error: 0.1076
Epoch 5/100
484/484 [==============================] - 9s 19ms/step - loss: 0.0847 - mean_absolute_error: 0.1007
Epoch 6/100
484/484 [==============================] - 9s 19ms/step - loss: 0.0864 - mean_absolute_error: 0.0983
Epoch 7/100
484/484 [==============================] - 9s 19ms/step - loss: 0.0875 - mean_absolute_error: 0.0979
Epoch 8/100
484/484 [==============================] - 9s 19ms/step - loss: 0.0892 - mean_absolute_error: 0.0944
Epoch 9/100
484/484 [==============================] - 9s 19ms/step - loss: 0.0834 - mean_absol

Сгенерируем прогнозы, сохраним в переменные X_train_pred, X_test_pred

In [ ]:
X_train_pred = vae.predict(X_train, batch_size=batch_size)
X_test_pred = vae.predict(X_test, batch_size=batch_size)

# 07. Метрика

Для возврата к исходному масштабу значений, используем ранее сохраненные среднее и стандартное отклонение временного ряда (переменные mean_x, stddev_x)

Для валидации модели, используем метрику median absolute error из библиотеки sklearn.

In [ ]:
def inverse_standartization(x):
    """Функция возвращающая значения к оригинальному масштабу
    x : временной ряд
    """
    global mean_x, stddev_x
    return (x * stddev_x) + mean_x

In [ ]:
# исходные данные
train_unscaled = inverse_standartization(X_train[:, 0, 0])
test_unscaled = inverse_standartization(X_test[:, 0, 0])

In [ ]:
# сгенерированные данные
train_pred_unscaled = inverse_standartization(X_train_pred[:, 0, 0])
test_pred_unscaled = inverse_standartization(X_test_pred[:, 0, 0])

Проверим значение целевой метрики на тестовой выборке.

In [ ]:
metric_medae = median_absolute_error(test_unscaled, test_pred_unscaled)
print("Median Absolute Error:", metric_medae, sep="\n")

Median Absolute Error:
7.067003110478453


Необходимая по точность модели (median absolute error < 10) достигнута.

# 08. Визуализация сгенерированных данных

Визуализируем сгенерированные данные с помощью библиотеки Plotly.

На графиках ниже представлены реальные и сгенерированные данные, а также вектор абсолютных ошибок между реальными и сгенерированными данными.

In [ ]:
def get_errors(real, pred):
    """Функция возвращающая вектор абсолютных ошибок
    real : исходный временной ряд
    pred : сгенерированный временной ряд    
    """
    return np.abs(real - pred)

In [ ]:
def get_result_plot(real, pred, name):
    """Функция выводящая графики реального и сгенерированного временного ряда
    real : исходный временной ряд
    pred : сгенерированный временной ряд
    name : наименование выборки
    """
    global timestamps
    # значения ошибок
    errors = get_errors(real, pred)
    
    # фигура для 2 графиков
    fig = subplots.make_subplots(
        rows=3, 
        cols=1,
        specs=[[{"rowspan": 2}],
               [None],
               [{}]],
        subplot_titles=(f"VAE generated time series from {name} marks",
                        f"Absolute errors predictions on {name} marks"))    
    # добавление графиков
    # реальные значения
    fig.add_trace(go.Scatter(x=timestamps, y=real, 
                             name=name, opacity=0.8),
                     row=1, col=1)
    # сгенерированные значения
    fig.add_trace(go.Scatter(x=timestamps, y=pred, 
                             name="Predict", opacity=0.6),
                     row=1, col=1)
    # абсолютные ошибки
    fig.add_trace(go.Scatter(x=timestamps, y=errors, 
                             name="Error", opacity=1),
                     row=3, col=1)                  
    # подпись осей
    fig.update_xaxes(title_text="Time", row=3, col=1)
    fig.update_yaxes(title_text="Marks", row=1, col=1)
    fig.update_yaxes(title_text="Errors", row=3, col=1)
    # установка размера и легенды
    fig.update_layout(showlegend=True, height=700)
    fig.show()

Первый график - данные сгенерированные вариационным автоэнкодером на тренировочной выборке.

In [ ]:
get_result_plot(real=train_unscaled, pred=train_pred_unscaled, name="Train")

Output hidden; open in https://colab.research.google.com to view.

Второй график - данные сгенерированные из тестовой выборки.

In [ ]:
get_result_plot(real=test_unscaled, pred=test_pred_unscaled, name="Test")

Output hidden; open in https://colab.research.google.com to view.

# 09. Заключение

- Проведена предобработка и визуализация данных.
- Разработана архитектура нейронной сети - вариационного автоэнкодера (VAE).
- Проведена валидация обученной нейронной сети. Необходимая точность достигнута.